# Creating a very simple muscular bioMod file
Now that we have decent kinematic and dynamic models, it would be useful to add some muscles to that muscle. 
The goal will be to convert the values obtained in inverse dynamics into muscle activation information which matches EMG (that can be measured in real life).
So as we did in `2-dynamic_model_creation` and in `2.5-simple_upper_limb` files, we will extend the models to add some muscles.

The strategy will be the same.
We load the previously created model and add extra information.

Please note that muscle is a vast subject in biomechanics and out of simplicity, we will use the simpliest muscle model that there is, which is that the muscle force is the maximal force of the muscle times its activation (it is called `IdealizedActuator` in `biorbd`).

In [ ]:
use_upper_limb = False 

# Prepare the name of the input and output file
if use_upper_limb:
    # Input
    dynamic_model_file_path = "models/SimpleUpperLimbWithInertia.bioMod"
    kinematic_chain_file_path = "models/upper_limb_kinematic_chain_with_inertia.kc"
    # Output
    muscular_model_file_path = "models/SimpleUpperLimbWithMuscles.bioMod"
    
    # Define a nice position to show the model
    rested_position = (1.20,)
    
else:
    # Input
    dynamic_model_file_path = "models/SimpleBodyWithInertia.bioMod"
    kinematic_chain_file_path = "models/simple_body_kinematic_chain_with_inertia.kc"
    # Output
    muscular_model_file_path = "models/SimpleBodyWithMuscles.bioMod"
    
    # Define a nice position to show the model
    rested_position = (0, 0, -0.15, 1.20, 0.7, -1, -1.11)

## Reload the dynamic model
In order ot load the kinematic chain (saved using pickle), we must define the classes that were used. 
So the next cell is a copy of the class declaration of `0-kinematic_model_creation`.

Please note that the actual KinematicChain class is skipped for now as we have to define first the Muscle class.

In [ ]:
# First of all, let's make our life easier by defining a class that we will fill for each of the segment
# Notice that mass, center_of_mass and inertia_xxyyzz are currently ignored. They will become relevant in 
# dynamics section below
class Segment():
    def __init__(
        self, 
        name, 
        parent_name="", 
        rt="",
        translations="", 
        rotations="",
        mass=0,
        center_of_mass=None,
        inertia_xxyyzz=None,
        mesh: tuple=None,
        markers: tuple=None,
    ):
        self.name = name
        self.parent_name = parent_name
        self.rt = rt
        self.translations = translations
        self.rotations = rotations
        self.mass = mass
        self.center_of_mass=center_of_mass
        self.inertia_xxyyzz=inertia_xxyyzz
        self.mesh = mesh
        self.markers = markers

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"segment {self.name}\n"
        if (self.parent_name):
            out_string += f"\tparent {self.parent_name}\n"
        if (self.rt):
            out_string += f"\tRT {self.rt}\n"
        if (self.translations):
            out_string += f"\ttranslations {self.translations}\n"
        if (self.rotations):
            out_string += f"\trotations {self.rotations}\n"
        if (self.mass):
            out_string += f"\tmass {self.mass}\n"
        if (self.center_of_mass):
            out_string += f"\tcom {self.center_of_mass[0]} {self.center_of_mass[1]} {self.center_of_mass[2]}\n"
        if (self.inertia_xxyyzz):
            out_string += f"\tinertia {self.inertia_xxyyzz[0]} 0 0\n" + \
                          f"\t        0 {self.inertia_xxyyzz[1]} 0\n" + \
                          f"\t        0 0 {self.inertia_xxyyzz[2]}\n"
        if (self.mesh):
            for m in self.mesh:
                out_string += f"\tmesh {m[0]} {m[1]} {m[2]}\n"
        out_string += "endsegment\n"
        
        # Also print the markers attached to the segment
        if (self.markers):
            for marker in self.markers:
                out_string += str(marker)
        return out_string
    
    
# Let's do the same for the markers
class Marker():
    def __init__(
        self, 
        name, 
        parent_name, 
        position,
    ):
        self.name = name
        self.parent_name = parent_name
        self.position = position

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"marker {self.name}\n"
        out_string += f"\tparent {self.parent_name}\n"
        out_string += f"\tposition {self.position[0]} {self.position[1]} {self.position[2]}\n"
        out_string += "endmarker\n"
        return out_string

## Extend with muscles
For that second part, we will add the muscles to the model. 
The way it works in `biorbd` is that a muscle is attached to a muscle group which is in turn attached to body parts (origin and insertion).

The muscles are therefore modelled as lines that are attached to a body part. 
They can be deviated by via points or wrapping objects. 
For the purpose of this tutorial, we limit ourselves to via points which are the easiest to deal with and are sufficient in most of the cases

In [ ]:
# So let's first declare these class so they can thereafter be added to the kinematic chain.

# This first class defines the name of the parent where the muscles are attached
class MuscleGroup:
    def __init__(self, name, origin, insertion):
        self.name = name
        self.origin = origin
        self.insertion = insertion
        self.muscles = []

    def add_muscle(self, name, origin, insertion, maximal_force, via_points=None):
        self.muscles.append(Muscle(name, self.name, origin, insertion, maximal_force, via_points))
        
    def __str__(self):
        out_string = f"musclegroup {self.name}\n"
        out_string += f"\tOriginParent {self.origin}\n"
        out_string += f"\tInsertionParent {self.insertion}\n"
        out_string += "endmusclegroup\n\n"
        for muscle in self.muscles:
            out_string += str(muscle)
        out_string += "\n"
        return out_string
        
# This second class is a blue print for each muscle
class Muscle:
    def __init__(self, name, muscle_group, origin, insertion, maximal_force, via_points=None):
        self.name = name
        self.type = "IdealizedActuator"
        self.muscle_group = muscle_group
        self.origin = origin
        self.insertion = insertion
        self.maximal_force = maximal_force
        self.via_points = via_points
        
    def __str__(self):
        out_string = f"\tmuscle {self.name}\n"
        out_string += f"\t\tType {self.type}\n"
        out_string += f"\t\tMuscleGroup {self.muscle_group}\n"
        out_string += f"\t\tOriginPosition {self.origin[0]} {self.origin[1]} {self.origin[2]}\n"
        out_string += f"\t\tInsertionPosition {self.insertion[0]} {self.insertion[1]} {self.insertion[2]}\n"
        out_string += f"\t\tMaximalForce {self.maximal_force}\n"
        out_string += "\tendmuscle\n\n"
        
        if self.via_points is not None:
            for i, via_point in enumerate(self.via_points):
                out_string += f"\t\tviapoint {self.name}_VIA-{i}\n"
                out_string += f"\t\t\tParent {via_point.parent}\n"
                out_string += f"\t\t\tMuscle {self.name}\n"
                out_string += f"\t\t\tMuscleGroup {self.muscle_group}\n"
                out_string += f"\t\t\tPosition {via_point.position[0]} {via_point.position[1]} {via_point.position[2]}\n"
                out_string += f"\t\tendviapoint\n\n"
        return out_string


# Finally this class is used for the via point Tuple sent to the Muscle constructor
class ViaPoint:
    def __init__(self, body_parent, position):
        self.parent = body_parent  # This is the body that will move this via point
        self.position = position  # The position in the reference frame of the body


In [ ]:
# Now we can define the KinematicChain class that includes the muscles and load the previously created kinematics chain (without the muscles yet)

# And why not creating a data structure that will collect all the segments and produce the a full
# kinematic chain that can easily be printed
class KinematicChain():
    def __init__(self, segments, muscle_groups=None):
        self.segments = segments
        self.muscle_groups = muscle_groups
        
    def __str__(self):
        out_string = "version 4\n\n"
        for segment in self.segments:
            out_string += str(segment)
            out_string += "\n\n\n"  # Give some space between segments
        
        for muscle_group in self.muscle_groups:
            out_string += str(muscle_group)
            out_string += "\n\n\n"  # Give some space between muscle groups
            
        return out_string
    
    def write(self, file_path):
        # Method to write the current KinematicChain to a file
        with open(file_path, "w") as file:
            file.write(str(self))
            
            
# So let's first reload the kinematic chain created in `2-dynamic_model_creation` or `2.5-simple_upper_limb` (saved using pickle)
# Meaning that after running this cell, the `kinematic_chain` variable is exactly what it was at the end of `2-dynamic_model_creation` or `2.5-simple_upper_limb`
import pickle

with open(kinematic_chain_file_path, "rb") as file:
    kinematic_chain = pickle.load(file)

# Collect the individual segments in the kinematic chain
trunk = kinematic_chain.segments[0]
head = kinematic_chain.segments[1]
upper_arm = kinematic_chain.segments[2]
lower_arm = kinematic_chain.segments[3]
hand = kinematic_chain.segments[4]
if use_upper_limb:
    # Recreate the kinematic chain using only upper part
    kinematic_chain = KinematicChain(segments=(trunk, head, upper_arm, lower_arm, hand))
else:
    thigh = kinematic_chain.segments[5]
    shank = kinematic_chain.segments[6]
    foot = kinematic_chain.segments[7]
    kinematic_chain = KinematicChain(segments=(trunk, head, upper_arm, lower_arm, hand, thigh, shank, foot))

In [ ]:
# Now let's add two muscles to the arm one that is producing the elevation (agonist) and one which is against (antagonist)
muscle_trunk_to_arm = MuscleGroup("TRUNK_TO_ARM", "TRUNK", "UPPER_ARM")

muscle_trunk_to_arm.add_muscle("BICEPS", (0, 0, 0.55), (0, 0.01, -0.25), 7000, (ViaPoint("UPPER_ARM", (0, 0.02, -0.1)), ))
muscle_trunk_to_arm.add_muscle("TRICEPS", (0, 0, 0.51), (0, -0.01, -0.25), 7000, (ViaPoint("UPPER_ARM", (0, -0.02, -0.1)), ))

kinematic_chain.muscle_groups = (muscle_trunk_to_arm, )

# Print and write the bioMod file with the muscles
# print(kinematic_chain)
kinematic_chain.write(muscular_model_file_path)

In [ ]:
# Run this cell to show the model you just created. 
# You should notice that the muscles now appear and also that the muscle module is now available
# Please note that due to some jupyter limitations, you may have to restart after running this cell
# Please also note that this cell will crash if runned from the binder

import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(muscular_model_file_path)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.set_q(rested_position)

# Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
viz.exec()

# If nothing happens, check for background. Sometimes the window loads behind the current window